<a href="https://colab.research.google.com/github/realmstoriches/my-ai-crew/blob/main/WVPowerBallPredictionApplication07042025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [141]:
import pandas as pd
import google.generativeai as genai
import os
from google.colab import userdata
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import sqlite3
import random
from collections import Counter
import itertools

# Configure the API key using Colab secrets
api_key = userdata.get('GOOGLE_API_KEY')
if not api_key:
    print("Error: GOOGLE_API_KEY not found in Colab secrets.")
    # Exit or handle the error appropriately if the API key is crucial for further execution
    # For this combined cell, we'll print an error and continue with limited functionality if the model is not initialized
genai.configure(api_key=api_key)

class Agent:
    def __init__(self, name, role_description):
        self.name = name
        self.role_description = role_description
        # Rely on genai.configure for API key
        try:
            # Change model name to a supported one
            self.model = genai.GenerativeModel('gemini-1.5-flash-latest')
        except Exception as e:
            print(f"Error initializing GenerativeModel for {self.name}: {e}")
            self.model = None # Set model to None if initialization fails

    def perform_task(self, prompt):
        if self.model:
            try:
                response = self.model.generate_content(prompt)
                return response.text
            except Exception as e:
                return f"Error generating content with Gemini model: {e}"
        else:
            return "Gemini model not initialized."


class ManagerAgent(Agent):
    def handle_task(self, task_description):
        print(f"{self.name} received task: {task_description}")

        # Simulate delegation to Data Scraping Agent
        print(f"{self.name} delegating to Data Scraping Agent...")
        # Hypothetical WV Lottery Powerball results URL - THIS NEEDS TO BE REPLACED
        # with the actual URL if available and accessible for scraping.
        # For this simulation, we will use a placeholder URL and acknowledge that actual scraping
        # might require more sophisticated techniques due to website structure or anti-scraping measures.
        wv_powerball_url = "https://www.wvlottery.com/games/powerball" # Placeholder URL - Updated to a potentially more relevant page
        scraped_data = agent_instances["Data Scraping Agent"].scrape_data(wv_powerball_url)
        print(f"Data Scraping Agent returned: {scraped_data.head() if isinstance(scraped_data, pd.DataFrame) else scraped_data}") # Print head if DataFrame

        if scraped_data is None or (isinstance(scraped_data, pd.DataFrame) and scraped_data.empty):
            print(f"{self.name}: Scraping failed or returned no data. Cannot proceed.")
            return "Task failed: Data scraping failed or returned no data."

        # Simulate delegation to Database Agent for storing raw data
        print(f"{self.name} delegating to Database Agent for raw data storage...")
        db_storage_status_raw = agent_instances["Database Agent"].store_data(scraped_data, table_name="raw_powerball_results")
        print(f"Database Agent (raw data) returned: {db_storage_status_raw}")

        if db_storage_status_raw.startswith("Error"):
            print(f"{self.name}: Storing raw data failed. Cannot proceed with cleaning.")
            return f"Task failed: {db_storage_status_raw}"


        # Simulate delegation to Data Cleaning Agent
        print(f"{self.name} delegating to Data Cleaning Agent...")
        # Retrieve raw data from the database for cleaning
        raw_data_from_db = agent_instances["Database Agent"].get_data("raw_powerball_results")

        if raw_data_from_db is None or raw_data_from_db.empty:
             print(f"{self.name}: Could not retrieve raw data from database for cleaning. Cannot proceed.")
             return "Task failed: Could not retrieve raw data from database."


        cleaned_data = agent_instances["Data Cleaning Agent"].clean_data(raw_data_from_db)
        print(f"Data Cleaning Agent returned: {cleaned_data.head() if isinstance(cleaned_data, pd.DataFrame) else cleaned_data}") # Print head if DataFrame

        if isinstance(cleaned_data, str) and cleaned_data.startswith("Error"):
             print(f"{self.name}: Data cleaning failed. Cannot proceed.")
             return f"Task failed: {cleaned_data}"

        # Simulate delegation to Database Agent for storing cleaned data
        print(f"{self.name} delegating to Database Agent for cleaned data storage...")
        db_storage_status_cleaned = agent_instances["Database Agent"].store_data(cleaned_data, table_name="cleaned_powerball_results")
        print(f"Database Agent (cleaned data) returned: {db_storage_status_cleaned}")

        if db_storage_status_cleaned.startswith("Error"):
            print(f"{self.name}: Storing cleaned data failed. Cannot proceed with modeling.")
            return f"Task failed: {db_storage_status_cleaned}"


        # Simulate delegation to Statistical Modeling Agent for initial analysis (frequency distribution)
        print(f"{self.name} delegating to Statistical Modeling Agent for initial analysis...")
        # Retrieve cleaned data from the database for analysis
        cleaned_data_from_db = agent_instances["Database Agent"].get_data("cleaned_powerball_results")

        if cleaned_data_from_db is None or cleaned_data_from_db.empty:
             print(f"{self.name}: Could not retrieve cleaned data from database for modeling. Cannot proceed.")
             return "Task failed: Could not retrieve cleaned data from database."

        analysis_results = agent_instances["Statistical Modeling Agent"].analyze_data(cleaned_data_from_db)
        print(f"Statistical Modeling Agent returned: {analysis_results}")

        if isinstance(analysis_results, str) and analysis_results.startswith("Error"):
             print(f"{self.name}: Statistical modeling failed. Cannot proceed.")
             return f"Task failed: {analysis_results}"


        # --- Re-evaluation Step ---
        print(f"\n{self.name} re-evaluating scenario...")
        print(f"{self.name} prompting Statistical Modeling Agent to generate ranked combinations...")

        # Call analyze_data again to get the ranked combinations
        ranked_combinations = agent_instances["Statistical Modeling Agent"].analyze_data(cleaned_data_from_db)

        if isinstance(ranked_combinations, str) and ranked_combinations.startswith("Error"):
             print(f"{self.name}: Statistical modeling re-evaluation failed. Cannot proceed with reporting.")
             return f"Task failed: Statistical modeling re-evaluation failed - {ranked_combinations}"

        print(f"Statistical Modeling Agent returned ranked combinations.")


        # Simulate delegation to Reporting Agent for generating the final report with top 10
        print(f"\n{self.name} delegating to Reporting Agent for final report generation with top 10 combinations...")
        final_report = agent_instances["Reporting Agent"].generate_report(ranked_combinations)
        print(f"Reporting Agent returned: {final_report}")


        return final_report


class DataScrapingAgent(Agent):
    def scrape_data(self, url):
        print(f"{self.name} is attempting to scrape data from: {url}")
        all_draws = []
        end_date = datetime.now()
        start_date = end_date - timedelta(days=5*365) # Approximate 5 years

        # Note: Actual scraping logic for historical data requires identifying
        # how the WV Lottery website provides historical results. This might involve:
        # 1. Finding a dedicated historical results page.
        # 2. Checking if results are paginated and iterating through pages.
        # 3. Looking for APIs or data feeds (less likely for a state lottery).
        # 4. Inspecting the page's HTML structure to locate draw dates and numbers.

        # The following is a *conceptual* scraping loop. It assumes a structure
        # where you might iterate through dates or pages.
        # Replace this with actual BeautifulSoup/requests logic based on the site.

        # --- Start of Conceptual Scraping Logic (Requires Replacement) ---
        # This is a simplified example and will NOT work without detailed knowledge
        # of the target website's HTML structure and URL patterns for historical data.
        # You would typically use requests.get(url) and then parse the HTML with BeautifulSoup.

        # Example placeholder data structure to mimic scraped results:
        # Each entry in all_draws would ideally come from parsing the website.
        # This section needs to be replaced with actual scraping code.

        print(f"{self.name}: Conceptual scraping simulation for 5 years.")
        # For demonstration, still using simulated data structure but acknowledging need for real scraping.
        # In a real scenario, this loop would fetch and parse data per draw or per page.
        simulated_data_points = 100 # Simulate more data points for better analysis
        for i in range(simulated_data_points):
             # Simulate generating data similar to the expected format
             draw_date = (start_date + timedelta(days=i*7)).strftime('%m/%d/%Y') # Simulate weekly draws
             white_balls = sorted([random.randint(1, 69) for _ in range(5)]) # Simulate 5 unique white balls
             powerball = random.randint(1, 26) # Simulate 1 powerball
             all_draws.append({'draw_date': draw_date, 'white_balls': white_balls, 'powerball': powerball})

        # --- End of Conceptual Scraping Logic ---


        if not all_draws:
            print(f"{self.name}: No data scraped (conceptual simulation returned empty).")
            return pd.DataFrame() # Return empty DataFrame if no data

        # Convert the collected data into a pandas DataFrame
        scraped_df = pd.DataFrame(all_draws)

        print(f"{self.name}: Conceptual scraping simulation completed. {len(scraped_df)} records generated.")
        return scraped_df

class DatabaseAgent(Agent):
     def __init__(self, name, role_description, db_connection):
        super().__init__(name, role_description)
        self.db_connection = db_connection
        self.cursor = self.db_connection.cursor()

     def store_data(self, data, table_name):
        """
        Stores data (pandas DataFrame) into the specified database table.

        Args:
            data: A pandas DataFrame containing the data to store.
            table_name: The name of the table to store the data in.

        Returns:
            A status message indicating success or failure.
        """
        print(f"{self.name} is storing data into table: {table_name}")

        if data is None or not isinstance(data, pd.DataFrame) or data.empty:
            print(f"{self.name}: Invalid or empty data received for storage.")
            return "Error: Invalid or empty data for storage."

        try:
            # Create table if it doesn't exist based on DataFrame columns
            # Using a more robust approach for column definitions
            cols_definitions = []
            for col, dtype in data.dtypes.items():
                 if dtype == 'object': # Assuming object might be lists or strings
                     cols_definitions.append(f"{col} TEXT")
                 elif dtype == 'int64':
                      cols_definitions.append(f"{col} INTEGER")
                 elif dtype == 'datetime64[ns]':
                      cols_definitions.append(f"{col} TEXT") # Store datetime as TEXT
                 else:
                      cols_definitions.append(f"{col} TEXT") # Default to TEXT

            columns_sql = ", ".join(cols_definitions)
            # Add a primary key for raw data to avoid duplicates on draw_date if it exists
            if 'draw_date' in data.columns and table_name == "raw_powerball_results":
                 create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_sql}, PRIMARY KEY (draw_date));"
            else:
                 create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_sql});"


            self.cursor.execute(create_table_query)
            self.db_connection.commit()
            print(f"{self.name}: Table '{table_name}' ensured to exist.")

            # Insert data into the table
            # This is a simplified insertion - a real implementation would handle data types
            # and potential conflicts more robustly.
            for index, row in data.iterrows():
                cols = ', '.join(row.index)
                vals = ', '.join(['?'] * len(row.values))
                insert_query = f"INSERT OR REPLACE INTO {table_name} ({cols}) VALUES ({vals})" # Use INSERT OR REPLACE to handle potential duplicates on primary key

                row_values = []
                for val in row.values.tolist():
                    if isinstance(val, list):
                        row_values.append(str(val)) # Convert lists to string for storage
                    elif isinstance(val, datetime):
                        row_values.append(val.strftime('%Y-%m-%d %H:%M:%S')) # Convert datetime to string
                    else:
                        row_values.append(val)


                self.cursor.execute(insert_query, row_values)

            self.db_connection.commit()
            print(f"{self.name}: Successfully stored {len(data)} records in '{table_name}'.")
            return f"Success: Stored {len(data)} records in '{table_name}'."

        except Exception as e:
            self.db_connection.rollback()
            print(f"{self.name}: Error storing data in '{table_name}': {e}")
            return f"Error: Could not store data in '{table_name}' - {e}"

     def get_data(self, table_name):
        """
        Retrieves data from the specified database table.

        Args:
            table_name: The name of the table to retrieve data from.

        Returns:
            A pandas DataFrame containing the retrieved data, or None if an error occurs.
        """
        print(f"{self.name} is retrieving data from table: {table_name}")
        try:
            query = f"SELECT * FROM {table_name};"
            self.cursor.execute(query)
            rows = self.cursor.fetchall()
            if not rows:
                print(f"{self.name}: No data found in table '{table_name}'.")
                return pd.DataFrame() # Return empty DataFrame if no data

            # Get column names from cursor description
            columns = [description[0] for description in self.cursor.description]

            df = pd.DataFrame(rows, columns=columns)

            # Attempt to convert 'white_balls' string back to list of integers if it exists
            if 'white_balls' in df.columns:
                try:
                    df['white_balls'] = df['white_balls'].apply(lambda x: [int(i) for i in x.strip('[]').split(',') if i.strip()])
                except Exception as e:
                    print(f"{self.name}: Warning: Could not convert 'white_balls' string to list: {e}")
                    # Keep as string if conversion fails

            # Attempt to convert 'draw_date' back to datetime if it exists
            if 'draw_date' in df.columns:
                try:
                    df['draw_date'] = pd.to_datetime(df['draw_date'])
                except Exception as e:
                    print(f"{self.name}: Warning: Could not convert 'draw_date' string to datetime: {e}")
                    # Keep as string if conversion fails


            print(f"{self.name}: Successfully retrieved {len(df)} records from '{table_name}'.")
            return df

        except Exception as e:
            print(f"{self.name}: Error retrieving data from '{table_name}': {e}")
            return None


class DataCleaningAgent(Agent):
    def clean_data(self, raw_data):
        """
        Cleans and preprocesses the raw Powerball data and extracts features.

        Args:
            raw_data: A pandas DataFrame containing the raw historical data.

        Returns:
            A pandas DataFrame with cleaned and formatted data and extracted features,
            or an error message.
        """
        print(f"{self.name} is cleaning data...")

        if raw_data is None or not isinstance(raw_data, pd.DataFrame):
            print(f"{self.name}: Invalid input data received.")
            return "Error: Invalid input data for cleaning."

        cleaned_df = raw_data.copy()

        # Ensure draw_date is in datetime format
        try:
            # It might already be datetime if retrieved from DB and converted in get_data
            if not pd.api.types.is_datetime64_any_dtype(cleaned_df['draw_date']):
                 cleaned_df['draw_date'] = pd.to_datetime(cleaned_df['draw_date'])
        except Exception as e:
            print(f"{self.name}: Error converting 'draw_date' to datetime: {e}")
            return "Error: Could not convert 'draw_date' to datetime."

        # Convert white_balls (list of integers) into separate columns
        try:
            # Ensure 'white_balls' is treated as a list
            if cleaned_df['white_balls'].apply(type).eq(str).any():
                 cleaned_df['white_balls'] = cleaned_df['white_balls'].apply(lambda x: [int(i) for i in x.strip('[]').split(',') if i.strip()])

            white_balls_df = pd.DataFrame(cleaned_df['white_balls'].tolist(), index=cleaned_df.index)
            white_balls_df.columns = [f'white_ball_{i+1}' for i in range(white_balls_df.shape[1])]
            cleaned_df = pd.concat([cleaned_df, white_balls_df], axis=1)
            cleaned_df = cleaned_df.drop('white_balls', axis=1) # Drop the original list column
        except Exception as e:
            print(f"{self.name}: Error processing 'white_balls' column: {e}")
            return "Error: Could not process 'white_balls' column."


        # Ensure powerball is of integer type
        try:
            cleaned_df['powerball'] = cleaned_df['powerball'].astype(int)
        except Exception as e:
            print(f"{self.name}: Error converting 'powerball' to integer: {e}")
            return "Error: Could not convert 'powerball' to integer."

        # Handle missing values (example: drop rows with any missing values - can be refined)
        initial_rows = len(cleaned_df)
        cleaned_df.dropna(inplace=True)
        if len(cleaned_df) < initial_rows:
            print(f"{self.name}: Dropped {initial_rows - len(cleaned_df)} rows with missing values.")

        # --- Feature Engineering ---

        # Calculate the sum of the five white balls
        white_ball_columns = [f'white_ball_{i+1}' for i in range(5)]
        cleaned_df['white_ball_sum'] = cleaned_df[white_ball_columns].sum(axis=1)

        # Calculate the count of odd and even white balls
        cleaned_df['odd_white_balls'] = cleaned_df[white_ball_columns].apply(lambda row: sum(x % 2 != 0 for x in row), axis=1)
        cleaned_df['even_white_balls'] = cleaned_df[white_ball_columns].apply(lambda row: sum(x % 2 == 0 for x in row), axis=1)

        # Check if any white balls were consecutive numbers
        def has_consecutive(numbers):
            sorted_numbers = sorted(numbers)
            for i in range(len(sorted_numbers) - 1):
                if sorted_numbers[i+1] == sorted_numbers[i] + 1:
                    return True
            return False

        cleaned_df['has_consecutive_white_balls'] = cleaned_df[white_ball_columns].apply(lambda row: has_consecutive(row.tolist()), axis=1)


        # Calculate the difference between the highest and lowest white ball
        cleaned_df['white_ball_range'] = cleaned_df[white_ball_columns].apply(lambda row: max(row) - min(row), axis=1)


        print(f"{self.name} finished cleaning data and extracting features. {len(cleaned_df)} records remaining.")
        return cleaned_df


class StatisticalModelingAgent(Agent):
    def analyze_data(self, cleaned_data):
        """
        Analyzes the cleaned Powerball data for patterns and anomalies and ranks combinations.

        Args:
            cleaned_data: A pandas DataFrame containing the cleaned historical data with engineered features.

        Returns:
            A ranked list of potential winning combinations (top 10), or an error message.
        """
        print(f"{self.name} is analyzing data for patterns and anomalies...")

        if cleaned_data is None or not isinstance(cleaned_data, pd.DataFrame) or cleaned_data.empty:
            print(f"{self.name}: Invalid or empty input data received for analysis.")
            return "Error: Invalid or empty input data for analysis."

        # 1. Calculate frequency of each individual number (white balls and powerball)
        white_ball_frequencies = Counter()
        powerball_frequencies = Counter()

        white_ball_columns = [f'white_ball_{i+1}' for i in range(5)]
        if not all(col in cleaned_data.columns for col in white_ball_columns):
             return "Error: White ball columns not found in the data."

        for col in white_ball_columns:
            white_ball_frequencies.update(cleaned_data[col])

        if 'powerball' in cleaned_data.columns:
             powerball_frequencies.update(cleaned_data['powerball'])
        else:
             return "Error: Powerball column not found for analysis."

        # 2. Analyze patterns based on engineered features
        # Analyze white ball sum distribution
        if 'white_ball_sum' not in cleaned_data.columns:
             return "Error: 'white_ball_sum' column not found. Ensure data cleaning created this feature."
        sum_distribution = cleaned_data['white_ball_sum'].value_counts(normalize=True).sort_index()

        # Analyze odd/even distribution
        if 'odd_white_balls' not in cleaned_data.columns or 'even_white_balls' not in cleaned_data.columns:
             return "Error: Odd/Even white ball columns not found. Ensure data cleaning created these features."
        odd_even_distribution = cleaned_data.groupby(['odd_white_balls', 'even_white_balls']).size().reset_index(name='count')
        odd_even_distribution['percentage'] = odd_even_distribution['count'] / len(cleaned_data)

        # Analyze consecutive numbers
        if 'has_consecutive_white_balls' not in cleaned_data.columns:
             return "Error: 'has_consecutive_white_balls' column not found. Ensure data cleaning created this feature."
        consecutive_percentage = cleaned_data['has_consecutive_white_balls'].value_counts(normalize=True).get(True, 0)

        # Analyze white ball range distribution
        if 'white_ball_range' not in cleaned_data.columns:
             return "Error: 'white_ball_range' column not found. Ensure data cleaning created this feature."
        range_distribution = cleaned_data['white_ball_range'].value_counts(normalize=True).sort_index()

        # 3. Develop a scoring/ranking mechanism
        def score_combination(combination, individual_freq, sum_dist, odd_even_dist, consecutive_perc, range_dist):
            white_balls = sorted(combination[:-1])
            powerball = combination[-1]

            # Score based on individual number frequencies
            individual_freq_score = sum(individual_freq.get(ball, 0) for ball in white_balls) + individual_freq.get(powerball, 0)

            # Score based on white ball sum
            current_sum = sum(white_balls)
            sum_score = sum_dist.get(current_sum, 0) # Use probability from distribution

            # Score based on odd/even distribution
            odd_count = sum(x % 2 != 0 for x in white_balls)
            even_count = sum(x % 2 == 0 for x in white_balls)
            odd_even_score_row = odd_even_dist[(odd_even_dist['odd_white_balls'] == odd_count) & (odd_even_dist['even_white_balls'] == even_count)]
            odd_even_score = odd_even_score_row['percentage'].iloc[0] if not odd_even_score_row.empty else 0

            # Score based on consecutive numbers (penalize if historical data shows few consecutive)
            has_consecutive = has_consecutive_in_list(white_balls)
            consecutive_score = consecutive_perc if has_consecutive else (1 - consecutive_perc) # Higher score if it matches the historical tendency

            # Score based on white ball range
            current_range = max(white_balls) - min(white_balls)
            range_score = range_dist.get(current_range, 0) # Use probability from distribution

            # Combine scores (weights can be adjusted)
            total_score = individual_freq_score + (sum_score * 100) + (odd_even_score * 100) + (consecutive_score * 50) + (range_score * 100) # Example weighting

            return total_score

        # Helper function for checking consecutive numbers in a list
        def has_consecutive_in_list(numbers):
            sorted_numbers = sorted(numbers)
            for i in range(len(sorted_numbers) - 1):
                if sorted_numbers[i+1] == sorted_numbers[i] + 1:
                    return True
            return False

        # 4. Generate potential combinations and rank them
        sample_size = 10000 # Reduced sample size for faster execution
        potential_combinations = []

        # Generate random combinations for scoring
        for _ in range(sample_size):
            white_balls = sorted(random.sample(range(1, 70), 5))
            powerball = random.randint(1, 26)
            potential_combinations.append(white_balls + [powerball])

        # Calculate scores for potential combinations
        scored_combinations = []
        for combo in potential_combinations:
            score = score_combination(combo, white_ball_frequencies, sum_distribution, odd_even_distribution, consecutive_percentage, range_distribution)
            scored_combinations.append((combo, score))

        # Rank combinations by score in descending order
        ranked_combinations = sorted(scored_combinations, key=lambda item: item[1], reverse=True)

        print(f"{self.name} finished analyzing data and ranking combinations.")

        # Return the top combinations (e.g., top 10)
        return ranked_combinations[:10]


class ReportingAgent(Agent):
    def generate_report(self, ranked_combinations):
        """
        Generates a report summarizing the top 10 ranked combinations.

        Args:
            ranked_combinations: A list of tuples, where each tuple contains a
                                 combination (list of numbers) and its score.

        Returns:
            A formatted report string.
        """
        print(f"{self.name} is generating report based on ranked combinations...")

        if not isinstance(ranked_combinations, list) or not ranked_combinations:
             return "Error: Invalid or empty ranked combinations data for reporting."

        # Format the ranked combinations for the report
        report_content = "## Top 10 Most Likely Powerball Combinations (Based on Historical Analysis)\n\n"
        report_content += "**IMPORTANT DISCLAIMER:** This report is based on historical data analysis and statistical patterns. Lottery draws are random events, and past results do not guarantee future outcomes. This analysis is for informational purposes only and should not be considered a prediction of future winning numbers. Playing the lottery should be for entertainment only, and responsible gambling is encouraged.\n\n"
        report_content += "Here are the top 10 combinations ranked by our analysis:\n\n"

        for i, (combination, score) in enumerate(ranked_combinations):
            white_balls_str = ', '.join(map(str, combination[:-1]))
            powerball_str = str(combination[-1])
            report_content += f"{i+1}. White Balls: {white_balls_str}, Powerball: {powerball_str} (Score: {score:.2f})\n"

        report_content += "\n---\n"
        report_content += "Analysis based on frequency distribution, white ball sum, odd/even distribution, consecutive numbers, and white ball range in historical West Virginia Powerball data."


        if self.model:
            # Use Gemini Pro to enhance the report
            prompt = f"""Review the following list of top 10 Powerball combinations generated by a statistical analysis of historical data. Provide a brief introductory and concluding paragraph for a report that presents these combinations. Ensure the report prominently includes the provided disclaimer.

Here are the top 10 combinations:
{report_content}
"""
            try:
                generated_content = self.model.generate_content(prompt).text
                return f"Simulated Report:\n{generated_content}\n\n{report_content}" # Combine generated text with the ranked list
            except Exception as e:
                print(f"{self.name}: Error generating content with Gemini model: {e}")
                return f"Simulated Report: Could not enhance report with Gemini model.\n\n{report_content}" # Return report without enhancement if API fails
        else:
            return f"Simulated Report: Gemini model not initialized.\n\n{report_content}" # Return report without enhancement if model not initialized


# Define the roles and responsibilities of each agent
roles = {
    "Manager": {
        "responsibilities": [
            "Delegate tasks to worker agents.",
            "Oversee the overall workflow.",
            "Ensure tasks are completed correctly and on time.",
            "Handle communication between agents.",
            "Report final results."
        ],
        "interactions": [
            "Receives requests from the user.",
            "Assigns tasks to specific worker agents.",
            "Receives status updates and results from worker agents."
        ]
    },
    "Data Scraping Agent": {
        "responsibilities": [
            "Scrape 5 years of historical Powerball winning numbers data from the WV lottery website.",
            "Handle website navigation and data extraction.",
            "Manage potential scraping issues (e.g., website structure changes, rate limits)."
        ],
        "interactions": [
            "Receives data scraping tasks from the Manager.",
            "Provides raw scraped data to the Database Agent or Manager."
        ]
    },
    "Database Agent": {
        "responsibilities": [
            "Store the collected historical Powerball data in a suitable database.",
            "Manage database connections and transactions.",
            "Ensure data integrity and accessibility."
        ],
        "interactions": [
            "Receives raw data from the Data Scraping Agent or Manager.",
            "Provides stored data to the Data Cleaning Agent or Manager."
        ]
    },
    "Data Cleaning Agent": {
        "responsibilities": [
            "Clean and preprocess the collected data.",
            "Handle missing values, outliers, and inconsistencies.",
            "Format the data for statistical modeling."
        ],
        "interactions": [
            "Receives raw data from the Data Collection Agent or Manager.",
            "Provides cleaned data to the Statistical Modeling Agent or Manager."
        ]
    },
    "Statistical Modeling Agent": {
        "responsibilities": [
            "Develop and train statistical models to predict winning numbers.",
            "Select appropriate modeling techniques (probability algorithm considering 1-69 for white balls and 1-26 for red ball).",
            "Evaluate model performance."
        ],
        "interactions": [
            "Receives trained models and prediction results to the Reporting Agent or Manager.",
            "Analyzes cleaned data from the Data Cleaning Agent or Manager." # Corrected interaction
        ]
    },
    "Reporting Agent": {
        "responsibilities": [
            "Generate reports summarizing the findings and predictions.",
            "Visualize data and model results.",
            "Present results in a clear and understandable format."
        ],
        "interactions": [
            "Receives model results and insights from the Statistical Modeling Agent or Manager.",
            "Provides final reports to the Manager."
        ]
    }
}

# Create an in-memory SQLite database connection
conn = sqlite3.connect(':memory:')

# Instantiate all agents with the correct classes
agent_instances = {}
for role, details in roles.items():
    agent_name = f"{role} Agent"
    role_description = ", ".join(details["responsibilities"])
    if role == "Manager":
        agent_instances[role] = ManagerAgent(agent_name, role_description)
    elif role == "Data Scraping Agent":
        agent_instances[role] = DataScrapingAgent(agent_name, role_description)
    elif role == "Database Agent":
        agent_instances[role] = DatabaseAgent(agent_name, role_description, conn) # Pass the database connection
    elif role == "Data Cleaning Agent":
        agent_instances[role] = DataCleaningAgent(agent_name, role_description)
    elif role == "Statistical Modeling Agent":
        agent_instances[role] = StatisticalModelingAgent(agent_name, role_description)
    elif role == "Reporting Agent":
        agent_instances[role] = ReportingAgent(agent_name, role_description)

# Define the hypothetical task
hypothetical_task = "Analyze historical Powerball winning numbers in West Virginia and generate a report on frequency distribution of numbers."
print(f"Hypothetical Task: {hypothetical_task}\n")

# Call the handle_task method of the ManagerAgent
final_simulated_report = agent_instances["Manager"].handle_task(hypothetical_task)

# Print a message indicating the simulation is complete and print the final report
print("\nSimulated agent interactions complete.")

# Print the final simulated report received by the Manager, which includes the Gemini generated content.
print("\n--- Enhanced Statistical Analysis Report ---")
print(final_simulated_report)

# Close the database connection when done (optional for in-memory, but good practice)
conn.close()

Hypothetical Task: Analyze historical Powerball winning numbers in West Virginia and generate a report on frequency distribution of numbers.

Manager Agent received task: Analyze historical Powerball winning numbers in West Virginia and generate a report on frequency distribution of numbers.
Manager Agent delegating to Data Scraping Agent...
Data Scraping Agent Agent is attempting to scrape data from: https://www.wvlottery.com/games/powerball
Data Scraping Agent Agent: Conceptual scraping simulation for 5 years.
Data Scraping Agent Agent: Conceptual scraping simulation completed. 100 records generated.
Data Scraping Agent returned:     draw_date           white_balls  powerball
0  07/05/2020  [28, 40, 50, 53, 59]         22
1  07/12/2020  [14, 28, 34, 42, 59]         26
2  07/19/2020   [8, 21, 24, 26, 36]          8
3  07/26/2020   [9, 36, 39, 43, 51]          1
4  08/02/2020   [1, 15, 26, 27, 48]         14
Manager Agent delegating to Database Agent for raw data storage...
Database Ag

In [140]:
import pandas as pd
import google.generativeai as genai
import os
from google.colab import userdata
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import sqlite3
import random
from collections import Counter
import itertools

# Configure the API key using Colab secrets
api_key = userdata.get('GOOGLE_API_KEY')
if not api_key:
    print("Error: GOOGLE_API_KEY not found in Colab secrets.")
    # Exit or handle the error appropriately if the API key is crucial for further execution
    # For this combined cell, we'll print an error and continue with limited functionality if the model is not initialized
genai.configure(api_key=api_key)

class Agent:
    def __init__(self, name, role_description):
        self.name = name
        self.role_description = role_description
        # Rely on genai.configure for API key
        try:
            # Change model name to a supported one
            self.model = genai.GenerativeModel('gemini-1.5-flash-latest')
        except Exception as e:
            print(f"Error initializing GenerativeModel for {self.name}: {e}")
            self.model = None # Set model to None if initialization fails

    def perform_task(self, prompt):
        if self.model:
            try:
                response = self.model.generate_content(prompt)
                return response.text
            except Exception as e:
                return f"Error generating content with Gemini model: {e}"
        else:
            return "Gemini model not initialized."


class ManagerAgent(Agent):
    def handle_task(self, task_description):
        print(f"{self.name} received task: {task_description}")

        # Simulate delegation to Data Scraping Agent
        print(f"{self.name} delegating to Data Scraping Agent...")
        # Hypothetical WV Lottery Powerball results URL - THIS NEEDS TO BE REPLACED
        # with the actual URL if available and accessible for scraping.
        # For this simulation, we will use a placeholder URL and acknowledge that actual scraping
        # might require more sophisticated techniques due to website structure or anti-scraping measures.
        wv_powerball_url = "https://www.wvlottery.com/games/powerball" # Placeholder URL - Updated to a potentially more relevant page
        scraped_data = agent_instances["Data Scraping Agent"].scrape_data(wv_powerball_url)
        print(f"Data Scraping Agent returned: {scraped_data.head() if isinstance(scraped_data, pd.DataFrame) else scraped_data}") # Print head if DataFrame

        if scraped_data is None or (isinstance(scraped_data, pd.DataFrame) and scraped_data.empty):
            print(f"{self.name}: Scraping failed or returned no data. Cannot proceed.")
            return "Task failed: Data scraping failed or returned no data."

        # Simulate delegation to Database Agent for storing raw data
        print(f"{self.name} delegating to Database Agent for raw data storage...")
        db_storage_status_raw = agent_instances["Database Agent"].store_data(scraped_data, table_name="raw_powerball_results")
        print(f"Database Agent (raw data) returned: {db_storage_status_raw}")

        if db_storage_status_raw.startswith("Error"):
            print(f"{self.name}: Storing raw data failed. Cannot proceed with cleaning.")
            return f"Task failed: {db_storage_status_raw}"


        # Simulate delegation to Data Cleaning Agent
        print(f"{self.name} delegating to Data Cleaning Agent...")
        # Retrieve raw data from the database for cleaning
        raw_data_from_db = agent_instances["Database Agent"].get_data("raw_powerball_results")

        if raw_data_from_db is None or raw_data_from_db.empty:
             print(f"{self.name}: Could not retrieve raw data from database for cleaning. Cannot proceed.")
             return "Task failed: Could not retrieve raw data from database."


        cleaned_data = agent_instances["Data Cleaning Agent"].clean_data(raw_data_from_db)
        print(f"Data Cleaning Agent returned: {cleaned_data.head() if isinstance(cleaned_data, pd.DataFrame) else cleaned_data}") # Print head if DataFrame

        if isinstance(cleaned_data, str) and cleaned_data.startswith("Error"):
             print(f"{self.name}: Data cleaning failed. Cannot proceed.")
             return f"Task failed: {cleaned_data}"

        # Simulate delegation to Database Agent for storing cleaned data
        print(f"{self.name} delegating to Database Agent for cleaned data storage...")
        db_storage_status_cleaned = agent_instances["Database Agent"].store_data(cleaned_data, table_name="cleaned_powerball_results")
        print(f"Database Agent (cleaned data) returned: {db_storage_status_cleaned}")

        if db_storage_status_cleaned.startswith("Error"):
            print(f"{self.name}: Storing cleaned data failed. Cannot proceed with modeling.")
            return f"Task failed: {db_storage_status_cleaned}"


        # Simulate delegation to Statistical Modeling Agent for initial analysis (frequency distribution)
        print(f"{self.name} delegating to Statistical Modeling Agent for initial analysis...")
        # Retrieve cleaned data from the database for analysis
        cleaned_data_from_db = agent_instances["Database Agent"].get_data("cleaned_powerball_results")

        if cleaned_data_from_db is None or cleaned_data_from_db.empty:
             print(f"{self.name}: Could not retrieve cleaned data from database for modeling. Cannot proceed.")
             return "Task failed: Could not retrieve cleaned data from database."

        analysis_results = agent_instances["Statistical Modeling Agent"].analyze_data(cleaned_data_from_db)
        print(f"Statistical Modeling Agent returned: {analysis_results}")

        if isinstance(analysis_results, str) and analysis_results.startswith("Error"):
             print(f"{self.name}: Statistical modeling failed. Cannot proceed.")
             return f"Task failed: {analysis_results}"


        # --- Re-evaluation Step ---
        print(f"\n{self.name} re-evaluating scenario...")
        print(f"{self.name} prompting Statistical Modeling Agent to generate ranked combinations...")

        # Call analyze_data again to get the ranked combinations
        ranked_combinations = agent_instances["Statistical Modeling Agent"].analyze_data(cleaned_data_from_db)

        if isinstance(ranked_combinations, str) and ranked_combinations.startswith("Error"):
             print(f"{self.name}: Statistical modeling re-evaluation failed. Cannot proceed with reporting.")
             return f"Task failed: Statistical modeling re-evaluation failed - {ranked_combinations}"

        print(f"Statistical Modeling Agent returned ranked combinations.")


        # Simulate delegation to Reporting Agent for generating the final report with top 10
        print(f"\n{self.name} delegating to Reporting Agent for final report generation with top 10 combinations...")
        final_report = agent_instances["Reporting Agent"].generate_report(ranked_combinations)
        print(f"Reporting Agent returned: {final_report}")


        return final_report


class DataScrapingAgent(Agent):
    def scrape_data(self, url):
        print(f"{self.name} is attempting to scrape data from: {url}")
        all_draws = []
        end_date = datetime.now()
        start_date = end_date - timedelta(days=5*365) # Approximate 5 years

        # Note: Actual scraping logic for historical data requires identifying
        # how the WV Lottery website provides historical results. This might involve:
        # 1. Finding a dedicated historical results page.
        # 2. Checking if results are paginated and iterating through pages.
        # 3. Looking for APIs or data feeds (less likely for a state lottery).
        # 4. Inspecting the page's HTML structure to locate draw dates and numbers.

        # The following is a *conceptual* scraping loop. It assumes a structure
        # where you might iterate through dates or pages.
        # Replace this with actual BeautifulSoup/requests logic based on the site.

        # --- Start of Conceptual Scraping Logic (Requires Replacement) ---
        # This is a simplified example and will NOT work without detailed knowledge
        # of the target website's HTML structure and URL patterns for historical data.
        # You would typically use requests.get(url) and then parse the HTML with BeautifulSoup.

        # Example placeholder data structure to mimic scraped results:
        # Each entry in all_draws would ideally come from parsing the website.
        # This section needs to be replaced with actual scraping code.

        print(f"{self.name}: Conceptual scraping simulation for 5 years.")
        # For demonstration, still using simulated data structure but acknowledging need for real scraping.
        # In a real scenario, this loop would fetch and parse data per draw or per page.
        simulated_data_points = 100 # Simulate more data points for better analysis
        for i in range(simulated_data_points):
             # Simulate generating data similar to the expected format
             draw_date = (start_date + timedelta(days=i*7)).strftime('%m/%d/%Y') # Simulate weekly draws
             white_balls = sorted([random.randint(1, 69) for _ in range(5)]) # Simulate 5 unique white balls
             powerball = random.randint(1, 26) # Simulate 1 powerball
             all_draws.append({'draw_date': draw_date, 'white_balls': white_balls, 'powerball': powerball})

        # --- End of Conceptual Scraping Logic ---


        if not all_draws:
            print(f"{self.name}: No data scraped (conceptual simulation returned empty).")
            return pd.DataFrame() # Return empty DataFrame if no data

        # Convert the collected data into a pandas DataFrame
        scraped_df = pd.DataFrame(all_draws)

        print(f"{self.name}: Conceptual scraping simulation completed. {len(scraped_df)} records generated.")
        return scraped_df

class DatabaseAgent(Agent):
     def __init__(self, name, role_description, db_connection):
        super().__init__(name, role_description)
        self.db_connection = db_connection
        self.cursor = self.db_connection.cursor()

     def store_data(self, data, table_name):
        """
        Stores data (pandas DataFrame) into the specified database table.

        Args:
            data: A pandas DataFrame containing the data to store.
            table_name: The name of the table to store the data in.

        Returns:
            A status message indicating success or failure.
        """
        print(f"{self.name} is storing data into table: {table_name}")

        if data is None or not isinstance(data, pd.DataFrame) or data.empty:
            print(f"{self.name}: Invalid or empty data received for storage.")
            return "Error: Invalid or empty data for storage."

        try:
            # Create table if it doesn't exist based on DataFrame columns
            # Using a more robust approach for column definitions
            cols_definitions = []
            for col, dtype in data.dtypes.items():
                 if dtype == 'object': # Assuming object might be lists or strings
                     cols_definitions.append(f"{col} TEXT")
                 elif dtype == 'int64':
                      cols_definitions.append(f"{col} INTEGER")
                 elif dtype == 'datetime64[ns]':
                      cols_definitions.append(f"{col} TEXT") # Store datetime as TEXT
                 else:
                      cols_definitions.append(f"{col} TEXT") # Default to TEXT

            columns_sql = ", ".join(cols_definitions)
            # Add a primary key for raw data to avoid duplicates on draw_date if it exists
            if 'draw_date' in data.columns and table_name == "raw_powerball_results":
                 create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_sql}, PRIMARY KEY (draw_date));"
            else:
                 create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_sql});"


            self.cursor.execute(create_table_query)
            self.db_connection.commit()
            print(f"{self.name}: Table '{table_name}' ensured to exist.")

            # Insert data into the table
            # This is a simplified insertion - a real implementation would handle data types
            # and potential conflicts more robustly.
            for index, row in data.iterrows():
                cols = ', '.join(row.index)
                vals = ', '.join(['?'] * len(row.values))
                insert_query = f"INSERT OR REPLACE INTO {table_name} ({cols}) VALUES ({vals})" # Use INSERT OR REPLACE to handle potential duplicates on primary key

                row_values = []
                for val in row.values.tolist():
                    if isinstance(val, list):
                        row_values.append(str(val)) # Convert lists to string for storage
                    elif isinstance(val, datetime):
                        row_values.append(val.strftime('%Y-%m-%d %H:%M:%S')) # Convert datetime to string
                    else:
                        row_values.append(val)


                self.cursor.execute(insert_query, row_values)

            self.db_connection.commit()
            print(f"{self.name}: Successfully stored {len(data)} records in '{table_name}'.")
            return f"Success: Stored {len(data)} records in '{table_name}'."

        except Exception as e:
            self.db_connection.rollback()
            print(f"{self.name}: Error storing data in '{table_name}': {e}")
            return f"Error: Could not store data in '{table_name}' - {e}"

     def get_data(self, table_name):
        """
        Retrieves data from the specified database table.

        Args:
            table_name: The name of the table to retrieve data from.

        Returns:
            A pandas DataFrame containing the retrieved data, or None if an error occurs.
        """
        print(f"{self.name} is retrieving data from table: {table_name}")
        try:
            query = f"SELECT * FROM {table_name};"
            self.cursor.execute(query)
            rows = self.cursor.fetchall()
            if not rows:
                print(f"{self.name}: No data found in table '{table_name}'.")
                return pd.DataFrame() # Return empty DataFrame if no data

            # Get column names from cursor description
            columns = [description[0] for description in self.cursor.description]

            df = pd.DataFrame(rows, columns=columns)

            # Attempt to convert 'white_balls' string back to list of integers if it exists
            if 'white_balls' in df.columns:
                try:
                    df['white_balls'] = df['white_balls'].apply(lambda x: [int(i) for i in x.strip('[]').split(',') if i.strip()])
                except Exception as e:
                    print(f"{self.name}: Warning: Could not convert 'white_balls' string to list: {e}")
                    # Keep as string if conversion fails

            # Attempt to convert 'draw_date' back to datetime if it exists
            if 'draw_date' in df.columns:
                try:
                    df['draw_date'] = pd.to_datetime(df['draw_date'])
                except Exception as e:
                    print(f"{self.name}: Warning: Could not convert 'draw_date' string to datetime: {e}")
                    # Keep as string if conversion fails


            print(f"{self.name}: Successfully retrieved {len(df)} records from '{table_name}'.")
            return df

        except Exception as e:
            print(f"{self.name}: Error retrieving data from '{table_name}': {e}")
            return None


class DataCleaningAgent(Agent):
    def clean_data(self, raw_data):
        """
        Cleans and preprocesses the raw Powerball data and extracts features.

        Args:
            raw_data: A pandas DataFrame containing the raw historical data.

        Returns:
            A pandas DataFrame with cleaned and formatted data and extracted features,
            or an error message.
        """
        print(f"{self.name} is cleaning data...")

        if raw_data is None or not isinstance(raw_data, pd.DataFrame):
            print(f"{self.name}: Invalid input data received.")
            return "Error: Invalid input data for cleaning."

        cleaned_df = raw_data.copy()

        # Ensure draw_date is in datetime format
        try:
            # It might already be datetime if retrieved from DB and converted in get_data
            if not pd.api.types.is_datetime64_any_dtype(cleaned_df['draw_date']):
                 cleaned_df['draw_date'] = pd.to_datetime(cleaned_df['draw_date'])
        except Exception as e:
            print(f"{self.name}: Error converting 'draw_date' to datetime: {e}")
            return "Error: Could not convert 'draw_date' to datetime."

        # Convert white_balls (list of integers) into separate columns
        try:
            # Ensure 'white_balls' is treated as a list
            if cleaned_df['white_balls'].apply(type).eq(str).any():
                 cleaned_df['white_balls'] = cleaned_df['white_balls'].apply(lambda x: [int(i) for i in x.strip('[]').split(',') if i.strip()])

            white_balls_df = pd.DataFrame(cleaned_df['white_balls'].tolist(), index=cleaned_df.index)
            white_balls_df.columns = [f'white_ball_{i+1}' for i in range(white_balls_df.shape[1])]
            cleaned_df = pd.concat([cleaned_df, white_balls_df], axis=1)
            cleaned_df = cleaned_df.drop('white_balls', axis=1) # Drop the original list column
        except Exception as e:
            print(f"{self.name}: Error processing 'white_balls' column: {e}")
            return "Error: Could not process 'white_balls' column."


        # Ensure powerball is of integer type
        try:
            cleaned_df['powerball'] = cleaned_df['powerball'].astype(int)
        except Exception as e:
            print(f"{self.name}: Error converting 'powerball' to integer: {e}")
            return "Error: Could not convert 'powerball' to integer."

        # Handle missing values (example: drop rows with any missing values - can be refined)
        initial_rows = len(cleaned_df)
        cleaned_df.dropna(inplace=True)
        if len(cleaned_df) < initial_rows:
            print(f"{self.name}: Dropped {initial_rows - len(cleaned_df)} rows with missing values.")

        # --- Feature Engineering ---

        # Calculate the sum of the five white balls
        white_ball_columns = [f'white_ball_{i+1}' for i in range(5)]
        cleaned_df['white_ball_sum'] = cleaned_df[white_ball_columns].sum(axis=1)

        # Calculate the count of odd and even white balls
        cleaned_df['odd_white_balls'] = cleaned_df[white_ball_columns].apply(lambda row: sum(x % 2 != 0 for x in row), axis=1)
        cleaned_df['even_white_balls'] = cleaned_df[white_ball_columns].apply(lambda row: sum(x % 2 == 0 for x in row), axis=1)

        # Check if any white balls were consecutive numbers
        def has_consecutive(numbers):
            sorted_numbers = sorted(numbers)
            for i in range(len(sorted_numbers) - 1):
                if sorted_numbers[i+1] == sorted_numbers[i] + 1:
                    return True
            return False

        cleaned_df['has_consecutive_white_balls'] = cleaned_df[white_ball_columns].apply(lambda row: has_consecutive(row.tolist()), axis=1)


        # Calculate the difference between the highest and lowest white ball
        cleaned_df['white_ball_range'] = cleaned_df[white_ball_columns].apply(lambda row: max(row) - min(row), axis=1)


        print(f"{self.name} finished cleaning data and extracting features. {len(cleaned_df)} records remaining.")
        return cleaned_df


class StatisticalModelingAgent(Agent):
    def analyze_data(self, cleaned_data):
        """
        Analyzes the cleaned Powerball data for patterns and anomalies and ranks combinations.

        Args:
            cleaned_data: A pandas DataFrame containing the cleaned historical data with engineered features.

        Returns:
            A ranked list of potential winning combinations (top 10), or an error message.
        """
        print(f"{self.name} is analyzing data for patterns and anomalies...")

        if cleaned_data is None or not isinstance(cleaned_data, pd.DataFrame) or cleaned_data.empty:
            print(f"{self.name}: Invalid or empty input data received for analysis.")
            return "Error: Invalid or empty input data for analysis."

        # 1. Calculate frequency of each individual number (white balls and powerball)
        white_ball_frequencies = Counter()
        powerball_frequencies = Counter()

        white_ball_columns = [f'white_ball_{i+1}' for i in range(5)]
        if not all(col in cleaned_data.columns for col in white_ball_columns):
             return "Error: White ball columns not found in the data."

        for col in white_ball_columns:
            white_ball_frequencies.update(cleaned_data[col])

        if 'powerball' in cleaned_data.columns:
             powerball_frequencies.update(cleaned_data['powerball'])
        else:
             return "Error: Powerball column not found for analysis."

        # 2. Analyze patterns based on engineered features
        # Analyze white ball sum distribution
        if 'white_ball_sum' not in cleaned_data.columns:
             return "Error: 'white_ball_sum' column not found. Ensure data cleaning created this feature."
        sum_distribution = cleaned_data['white_ball_sum'].value_counts(normalize=True).sort_index()

        # Analyze odd/even distribution
        if 'odd_white_balls' not in cleaned_data.columns or 'even_white_balls' not in cleaned_data.columns:
             return "Error: Odd/Even white ball columns not found. Ensure data cleaning created these features."
        odd_even_distribution = cleaned_data.groupby(['odd_white_balls', 'even_white_balls']).size().reset_index(name='count')
        odd_even_distribution['percentage'] = odd_even_distribution['count'] / len(cleaned_data)

        # Analyze consecutive numbers
        if 'has_consecutive_white_balls' not in cleaned_data.columns:
             return "Error: 'has_consecutive_white_balls' column not found. Ensure data cleaning created this feature."
        consecutive_percentage = cleaned_data['has_consecutive_white_balls'].value_counts(normalize=True).get(True, 0)

        # Analyze white ball range distribution
        if 'white_ball_range' not in cleaned_data.columns:
             return "Error: 'white_ball_range' column not found. Ensure data cleaning created this feature."
        range_distribution = cleaned_data['white_ball_range'].value_counts(normalize=True).sort_index()

        # 3. Develop a scoring/ranking mechanism
        def score_combination(combination, individual_freq, sum_dist, odd_even_dist, consecutive_perc, range_dist):
            white_balls = sorted(combination[:-1])
            powerball = combination[-1]

            # Score based on individual number frequencies
            individual_freq_score = sum(individual_freq.get(ball, 0) for ball in white_balls) + individual_freq.get(powerball, 0)

            # Score based on white ball sum
            current_sum = sum(white_balls)
            sum_score = sum_dist.get(current_sum, 0) # Use probability from distribution

            # Score based on odd/even distribution
            odd_count = sum(x % 2 != 0 for x in white_balls)
            even_count = sum(x % 2 == 0 for x in white_balls)
            odd_even_score_row = odd_even_dist[(odd_even_dist['odd_white_balls'] == odd_count) & (odd_even_dist['even_white_balls'] == even_count)]
            odd_even_score = odd_even_score_row['percentage'].iloc[0] if not odd_even_score_row.empty else 0

            # Score based on consecutive numbers (penalize if historical data shows few consecutive)
            has_consecutive = has_consecutive_in_list(white_balls)
            consecutive_score = consecutive_perc if has_consecutive else (1 - consecutive_perc) # Higher score if it matches the historical tendency

            # Score based on white ball range
            current_range = max(white_balls) - min(white_balls)
            range_score = range_dist.get(current_range, 0) # Use probability from distribution

            # Combine scores (weights can be adjusted)
            total_score = individual_freq_score + (sum_score * 100) + (odd_even_score * 100) + (consecutive_score * 50) + (range_score * 100) # Example weighting

            return total_score

        # Helper function for checking consecutive numbers in a list
        def has_consecutive_in_list(numbers):
            sorted_numbers = sorted(numbers)
            for i in range(len(sorted_numbers) - 1):
                if sorted_numbers[i+1] == sorted_numbers[i] + 1:
                    return True
            return False

        # 4. Generate potential combinations and rank them
        sample_size = 10000 # Reduced sample size for faster execution
        potential_combinations = []

        # Generate random combinations for scoring
        for _ in range(sample_size):
            white_balls = sorted(random.sample(range(1, 70), 5))
            powerball = random.randint(1, 26)
            potential_combinations.append(white_balls + [powerball])

        # Calculate scores for potential combinations
        scored_combinations = []
        for combo in potential_combinations:
            score = score_combination(combo, white_ball_frequencies, sum_distribution, odd_even_distribution, consecutive_percentage, range_distribution)
            scored_combinations.append((combo, score))

        # Rank combinations by score in descending order
        ranked_combinations = sorted(scored_combinations, key=lambda item: item[1], reverse=True)

        print(f"{self.name} finished analyzing data and ranking combinations.")

        # Return the top combinations (e.g., top 10)
        return ranked_combinations[:10]


class ReportingAgent(Agent):
    def generate_report(self, ranked_combinations):
        """
        Generates a report summarizing the top 10 ranked combinations.

        Args:
            ranked_combinations: A list of tuples, where each tuple contains a
                                 combination (list of numbers) and its score.

        Returns:
            A formatted report string.
        """
        print(f"{self.name} is generating report based on ranked combinations...")

        if not isinstance(ranked_combinations, list) or not ranked_combinations:
             return "Error: Invalid or empty ranked combinations data for reporting."

        # Format the ranked combinations for the report
        report_content = "## Top 10 Most Likely Powerball Combinations (Based on Historical Analysis)\n\n"
        report_content += "**IMPORTANT DISCLAIMER:** This report is based on historical data analysis and statistical patterns. Lottery draws are random events, and past results do not guarantee future outcomes. This analysis is for informational purposes only and should not be considered a prediction of future winning numbers. Playing the lottery should be for entertainment only, and responsible gambling is encouraged.\n\n"
        report_content += "Here are the top 10 combinations ranked by our analysis:\n\n"

        for i, (combination, score) in enumerate(ranked_combinations):
            white_balls_str = ', '.join(map(str, combination[:-1]))
            powerball_str = str(combination[-1])
            report_content += f"{i+1}. White Balls: {white_balls_str}, Powerball: {powerball_str} (Score: {score:.2f})\n"

        report_content += "\n---\n"
        report_content += "Analysis based on frequency distribution, white ball sum, odd/even distribution, consecutive numbers, and white ball range in historical West Virginia Powerball data."


        if self.model:
            # Use Gemini Pro to enhance the report
            prompt = f"""Review the following list of top 10 Powerball combinations generated by a statistical analysis of historical data. Provide a brief introductory and concluding paragraph for a report that presents these combinations. Ensure the report prominently includes the provided disclaimer.

Here are the top 10 combinations:
{report_content}
"""
            try:
                generated_content = self.model.generate_content(prompt).text
                return f"Simulated Report:\n{generated_content}\n\n{report_content}" # Combine generated text with the ranked list
            except Exception as e:
                print(f"{self.name}: Error generating content with Gemini model: {e}")
                return f"Simulated Report: Could not enhance report with Gemini model.\n\n{report_content}" # Return report without enhancement if API fails
        else:
            return f"Simulated Report: Gemini model not initialized.\n\n{report_content}" # Return report without enhancement if model not initialized


# Define the roles and responsibilities of each agent
roles = {
    "Manager": {
        "responsibilities": [
            "Delegate tasks to worker agents.",
            "Oversee the overall workflow.",
            "Ensure tasks are completed correctly and on time.",
            "Handle communication between agents.",
            "Report final results."
        ],
        "interactions": [
            "Receives requests from the user.",
            "Assigns tasks to specific worker agents.",
            "Receives status updates and results from worker agents."
        ]
    },
    "Data Scraping Agent": {
        "responsibilities": [
            "Scrape 5 years of historical Powerball winning numbers data from the WV lottery website.",
            "Handle website navigation and data extraction.",
            "Manage potential scraping issues (e.g., website structure changes, rate limits)."
        ],
        "interactions": [
            "Receives data scraping tasks from the Manager.",
            "Provides raw scraped data to the Database Agent or Manager."
        ]
    },
    "Database Agent": {
        "responsibilities": [
            "Store the collected historical Powerball data in a suitable database.",
            "Manage database connections and transactions.",
            "Ensure data integrity and accessibility."
        ],
        "interactions": [
            "Receives raw data from the Data Scraping Agent or Manager.",
            "Provides stored data to the Data Cleaning Agent or Manager."
        ]
    },
    "Data Cleaning Agent": {
        "responsibilities": [
            "Clean and preprocess the collected data.",
            "Handle missing values, outliers, and inconsistencies.",
            "Format the data for statistical modeling."
        ],
        "interactions": [
            "Receives raw data from the Data Collection Agent or Manager.",
            "Provides cleaned data to the Statistical Modeling Agent or Manager."
        ]
    },
    "Statistical Modeling Agent": {
        "responsibilities": [
            "Develop and train statistical models to predict winning numbers.",
            "Select appropriate modeling techniques (probability algorithm considering 1-69 for white balls and 1-26 for red ball).",
            "Evaluate model performance."
        ],
        "interactions": [
            "Receives trained models and prediction results to the Reporting Agent or Manager.",
            "Analyzes cleaned data from the Data Cleaning Agent or Manager." # Corrected interaction
        ]
    },
    "Reporting Agent": {
        "responsibilities": [
            "Generate reports summarizing the findings and predictions.",
            "Visualize data and model results.",
            "Present results in a clear and understandable format."
        ],
        "interactions": [
            "Receives model results and insights from the Statistical Modeling Agent or Manager.",
            "Provides final reports to the Manager."
        ]
    }
}

# Create an in-memory SQLite database connection
conn = sqlite3.connect(':memory:')

# Instantiate all agents with the correct classes
agent_instances = {}
for role, details in roles.items():
    agent_name = f"{role} Agent"
    role_description = ", ".join(details["responsibilities"])
    if role == "Manager":
        agent_instances[role] = ManagerAgent(agent_name, role_description)
    elif role == "Data Scraping Agent":
        agent_instances[role] = DataScrapingAgent(agent_name, role_description)
    elif role == "Database Agent":
        agent_instances[role] = DatabaseAgent(agent_name, role_description, conn) # Pass the database connection
    elif role == "Data Cleaning Agent":
        agent_instances[role] = DataCleaningAgent(agent_name, role_description)
    elif role == "Statistical Modeling Agent":
        agent_instances[role] = StatisticalModelingAgent(agent_name, role_description)
    elif role == "Reporting Agent":
        agent_instances[role] = ReportingAgent(agent_name, role_description)

# Define the hypothetical task
hypothetical_task = "Analyze historical Powerball winning numbers in West Virginia and generate a report on frequency distribution of numbers."
print(f"Hypothetical Task: {hypothetical_task}\n")

# Call the handle_task method of the ManagerAgent
final_simulated_report = agent_instances["Manager"].handle_task(hypothetical_task)

# Print a message indicating the simulation is complete and print the final report
print("\nSimulated agent interactions complete.")

# Print the final simulated report received by the Manager, which includes the Gemini generated content.
print("\n--- Enhanced Statistical Analysis Report ---")
print(final_simulated_report)

# Close the database connection when done (optional for in-memory, but good practice)
conn.close()

Hypothetical Task: Analyze historical Powerball winning numbers in West Virginia and generate a report on frequency distribution of numbers.

Manager Agent received task: Analyze historical Powerball winning numbers in West Virginia and generate a report on frequency distribution of numbers.
Manager Agent delegating to Data Scraping Agent...
Data Scraping Agent Agent is attempting to scrape data from: https://www.wvlottery.com/games/powerball
Data Scraping Agent Agent: Conceptual scraping simulation for 5 years.
Data Scraping Agent Agent: Conceptual scraping simulation completed. 100 records generated.
Data Scraping Agent returned:     draw_date           white_balls  powerball
0  07/05/2020  [11, 18, 47, 59, 61]         17
1  07/12/2020   [9, 16, 38, 60, 64]         26
2  07/19/2020  [15, 41, 57, 59, 65]         20
3  07/26/2020  [12, 16, 29, 40, 60]         23
4  08/02/2020  [15, 23, 42, 65, 69]         14
Manager Agent delegating to Database Agent for raw data storage...
Database Ag

## Summary:

### Insights or Next Steps
*   The current statistical model is a simplified approach. Future work could explore more advanced statistical or machine learning techniques to potentially identify more subtle historical patterns, while always maintaining the disclaimer about true predictability.
*   The data scraping component is currently conceptual. A critical next step for a real-world application would be to develop robust and reliable data scraping logic for the West Virginia Lottery website, ensuring compliance with their terms of service.
